# Gold Aggregation

This notebook aggregates curated (silver) data into gold-level metrics.


In [1]:
try:
    spark
except NameError:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.appName("gold-aggregation").getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/06 14:25:22 WARN Utils: Your hostname, MA-L-481079, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/06 14:25:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/06 14:25:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/06 14:25:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/06 14:25:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
from pyspark.sql import functions as F

data_path = "../../data/example.csv"

raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(data_path)
)

silver = (
    raw
    .withColumn("signup_date", F.to_date("signup_date"))
    .withColumn("spend", F.col("spend").cast("double"))
)


In [4]:
gold_by_plan = (
    silver
    .groupBy("plan")
    .agg(
        F.count("*").alias("users"),
        F.round(F.sum("spend"), 2).alias("total_spend"),
        F.round(F.avg("spend"), 2).alias("avg_spend"),
        F.sum(F.when(F.col("is_active"), 1).otherwise(0)).alias("active_users"),
    )
    .orderBy(F.col("total_spend").desc())
)

gold_by_plan.show(truncate=False)


+----------+-----+-----------+---------+------------+
|plan      |users|total_spend|avg_spend|active_users|
+----------+-----+-----------+---------+------------+
|Enterprise|2    |3540.1     |1770.05  |2           |
|Pro       |3    |1099.75    |366.58   |3           |
|Basic     |3    |148.99     |49.66    |2           |
|Free      |2    |12.75      |6.38     |1           |
+----------+-----+-----------+---------+------------+



In [5]:
gold_by_month = (
    silver
    .withColumn("signup_month", F.date_format("signup_date", "yyyy-MM"))
    .groupBy("signup_month")
    .agg(
        F.count("*").alias("users"),
        F.round(F.sum("spend"), 2).alias("total_spend"),
    )
    .orderBy("signup_month")
)

gold_by_month.show(truncate=False)


+------------+-----+-----------+
|signup_month|users|total_spend|
+------------+-----+-----------+
|2020-08     |1    |2300.1     |
|2021-03     |1    |1240.0     |
|2022-02     |1    |410.25     |
|2022-06     |1    |560.0      |
|2023-11     |1    |0.0        |
|2023-12     |1    |0.0        |
|2024-01     |1    |129.5      |
|2024-05     |1    |89.99      |
|2024-07     |1    |59.0       |
|2024-09     |1    |12.75      |
+------------+-----+-----------+

